# Image Clustering Notebook
The notebook is to use K-means cluster for unsupervised classificaiton on multi-band raster data <br>
Folium serves as visualization backend <br>

<br>
Basic workflow: <br>
<ul>
    <li> Read raster data </li>
    <li> Plot raster data </li>
    <li> Run clustering </li>
    <li> Plot result with raster </li>
    <li> Re-label clusters </li>
    <li> Plot updated result </li>
    <li> Write out clustering labels </li>
</ul>

<br>

<i>version 1.0  Date: Mar-2022</i>

<br>
<i>version 1.1  Date: Apr.-2022:
<ul> 
    <li> Use geoViews for vis </li>
    <li> Add widget to select lables </li> 
<i/>


In [ ]:
import rasterio as rio
import rioxarray as rxr
import xarray as xr
from sklearn import cluster
import numpy as np

from rasterio.plot import reshape_as_image
import geoviews as gv

import holoviews.operation.datashader as hd

import ipywidgets as widgets
import panel as pn
pn.extension()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#using bokeh backend
gv.extension('bokeh')

In [ ]:
from cartopy import crs
from joblib import cpu_count

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
def get_cluster(raster_file, bands=None, n_cluster=5):
    src = rio.open(raster_file)
    if bands:
        arr = src.read(bands)
    else:
        arr = src.read()
    
    img = reshape_as_image(arr)
    img1d = img.reshape(-1, img.shape[-1])
    
    #parameters for KMeans
    params = {
    'n_clusters' : n_cluster,
    'random_state' : 0,
    'batch_size' : 256*cpu_count()
    }
    cl = cluster.MiniBatchKMeans(**params)
    model = cl.fit(img1d)
    # 
    img_cl = model.labels_
    img_cl = img_cl.reshape(img[:,:,0].shape)
    return img_cl


def relabel(lab_arr, lookup):
    new_lab = lab_arr.copy()
    for k, v in lookup.items():
        if len(v)==1 & k==v[0]:
            pass
        else:
            new_lab = np.where(np.isin(new_lab, v), k, new_lab)
    return new_lab

def np2ds(x, y, arr):
    # load 2-D np.array to gv.dataset
    # x: vector          x-coord (longitude)
    # y: vector          y-coord (latitude)
    # arr: 2-D Array     shape(x,y)
    # returns:
    #     min(arr), max(arr), gv.DataSet
    return arr.min(), arr.max(), gv.Dataset((x, y, arr), ['longitude', 'latitude'], 'class')

In [ ]:
from datashader.utils import ngjit

@ngjit
def normalize_data(agg):
    out = np.zeros_like(agg)
    ii = np.iinfo(agg.dtype)
    min_val = ii.min
    max_val = ii.max-1
    range_val = max_val-min_val
    
    cols, rows = agg.shape
    c = 40
    th = .125
    for x in range(cols):
        for y in range(rows):
            val = agg[x, y]
            norm = (val-min_val)/range_val
            #norm = 1/(1+np.exp(c*(th-norm)))
            out[x,y] = norm
    return out

In [ ]:
# path to raster
rfile = "landsat.tif"

# load raster with rioxarry, can be replaced by other libs
ds = rxr.open_rasterio(rfile)

# show raster profile & spatial reference
print(ds)
print(ds.rio.crs)

## Show raster map

In [ ]:
# specify rgb bands 
rb, gb, bb = (5,4,3)

# extract bands values 
r,g,b = map(np.array, [ds.sel(band=rb).squeeze(), ds.sel(band=gb).squeeze(), ds.sel(band=bb).squeeze()])

# normalize if necessary
# r = normalize_data(r).astype(np.uint8)
# g = normalize_data(g).astype(np.unit8)
# b = normalize_data(b).astype(np.uinit8)


In [ ]:
# set basemap
# other available basemap samples  https://geoviews.org/user_guide/Working_with_Bokeh.html 
base_map = gv.tile_sources.EsriImagery.opts(width=800, height=800)

In [ ]:
# set options for raster map
img_opts = dict(width=600, height=600)
raster_layer = gv.RGB((ds.x, ds.y, r, g, b), vdims=list('RGB'), crs=crs.PlateCarree()).opts(**img_opts)

In [ ]:
# web rendering is slow for large image
pn.Column(raster_layer*base_map)

## Run K-Means Clustering for Image
User can run clustering for subset of bands by giving a list; If not specified, all bands will be included  

In [ ]:
%%time
# get_cluster takes dataset object (not np.array) 
# and (optional) list of bands as inputs 
labels = get_cluster(rfile, bands=(5,4,3), n_cluster=30)
labels.shape

## Show cluster map


In [ ]:

# load data into a Dataset
minv, maxv, cls_ds = np2ds(ds.x, ds.y, labels)

# set options for cluster map
cls_opts = dict(
    width=600, 
    height=600, 
    cmap="Category20b",
    colorbar=False,
    clim = (minv, maxv),
    
    tools=["hover"], active_tools=['wheel_zoom']
)
cluster_layer = gv.Image(cls_ds, crs=crs.PlateCarree())

pn.Column(cluster_layer.opts(**cls_opts)*base_map.opts(height=600, width=600))

## Display maps side-by-side 


In [ ]:
import time
s = time.time()
pn.Column(cluster_layer*raster_layer*base_map + cluster_layer*base_map)


## Update labels 
User can aggregate classes into new groups through loop-up table

In [ ]:
tab = {}
def update_dict(op):
    if op == 'N':
        key = list(sl.value)[0]
        tab[key] = list(sl.value)
        print("Re-grouping : ", tab)
    if op == 'D':
        if len(sl.options) > 0:
            key = list(sl.options)[0]
            tab[key] = list(sl.options)
        print("Final Groups : ", tab)
    if op == 'S':
        tab.clear()
        print("Start Over : ", tab)
    
def update_list(old, out):
    return [ele for ele in old if ele not in out]

print("Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.")
opts = list(np.unique(labels))
sl = widgets.SelectMultiple(options=opts, layout=(widgets.Layout(height='200px', width='150px')))
bt = widgets.ToggleButtons(options=['Select:', 'Next', 'Done', 'Start Over'], value='Select:')
output = widgets.Output()
display(sl, bt, output)
def handle_click(change):
    with output:
        if change.new == "Next":
            nn = update_list(list(sl.options), list(sl.value))
            update_dict("N")
            sl.options = nn
            bt.value='Select:'
        if change.new == "Done":
            update_dict("D")
        if change.new == "Start Over":
            sl.options = opts
            update_dict("S")
            bt.value = 'Select:'
            

            #sl.value = n[-1]

bt.observe(handle_click, names='value')

In [ ]:
input("> Press Enter to Continue...\n")

# This cell updates clusters with new labels & converts labels into sequential one
new_labels = relabel(labels, tab)

t = {}
for i, v in enumerate(np.unique(new_labels)):
    t[i] = [v]

new_labels_seq = relabel(new_labels, t)
print("New Classes", np.unique(new_labels_seq))

In [ ]:
# load data into a Dataset
minv, maxv, cls_ds_new = np2ds(ds.x, ds.y, new_labels_seq)

# set options for cluster map

cls_opts = dict(
    width=600, 
    height=600, 
    cmap="Category20b",
    colorbar=False,
    clim = (minv, maxv),   
    tools=["hover"], active_tools=['wheel_zoom']
)

cluster_layer_new = gv.Image(cls_ds_new, crs=crs.PlateCarree()).opts(**cls_opts)


In [ ]:
pn.Column(cluster_layer_new*raster_layer*base_map+cluster_layer_new*base_map)

## Export labels to geoTiff

In [ ]:
# path to output file
save_path = "./landsat_cluster.tif"

# labels to write out
rslt = new_labels_seq

In [ ]:
'''
# using profile from input raster
with rio.open(rfile) as src:
    profile = src.meta
    # update number of bands
    profile.update(count=1)
    
    with rio.open(save_path, 'w', **profile) as dst:
        ds.write(rslt, indexes = 1)
'''

In [ ]:
'''
# (optional) double check write-out labels
with rio.open(save_path) as ck:
    print(ck.meta)
    print(np.unique(ck.read()))
'''